# Open AI Quote Generator (Function Calling)

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>

## Description:

A simple quote generator that leverages OpenAI’s GPT model to generate unique quotes on random topics. The app ensures the generated quotes are not duplicated by storing them in a local database and checking their uniqueness using OpenAI’s function calling mechanism. It also provides a seamless experience for installing dependencies, setting up the environment, and managing the quote database.


## Step 1: Install Requirements

### Purpose:

This step ensures that all the necessary Python dependencies are installed for the application to run smoothly.

### Code Logic:

- The `install_requirements()` function checks if the requirements have already been installed by checking the `requirements_installed` flag.

- If not, it attempts to install the dependencies by executing the `pip install -r requirements.txt` command using `os.system()`.

- It also includes retry logic to retry the installation process up to 3 times in case of failure.

- If the installation fails after the retries, the process exits.


In [7]:
import os

requirements_installed = False
max_retries = 3
retries = 0


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return

install_requirements()

## Step 2: Set Up Environment Variables

### Purpose:

This step loads environment variables, particularly the `OPENAI_API_KEY`, necessary to interact with OpenAI's API.

### Code Logic:

- The `setup_env()` function calls `load_dotenv()` to load environment variables from a `.env` file.

- It checks if the `OPENAI_API_KEY` environment variable is set using `os.getenv()`. If not, it will print a message and exit the program.

- If the API key is set, it prints a confirmation message.


In [3]:
from dotenv import load_dotenv
import os


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv()

    variables_to_check = ["OPENAI_API_KEY"]

    for var in variables_to_check:
        check_env(var)

setup_env()

## Step 3: Define QUOTABLE_TOPICS

### Purpose:

This step defines a list of topics from which the quote generator can generate quotes.

### Code Logic:

- The `QUOTABLE_TOPICS` list contains various topics like `"inspirational"`, `"motivational"`, `"love"`, `"happiness"`, and many more.

- This list will be used later to randomly select a topic for generating quotes.


In [6]:
QUOTABLE_TOPICS = topics = [
    "inspirational",
    "motivational",
    "life",
    "love",
    "success",
    "happiness",
    "wisdom",
    "leadership",
    "friendship",
    "education",
    "family",
    "peace",
    "hope",
    "dreams",
    "creativity",
    "imagination",
    "confidence",
    "self-love",
    "self-care",
    "self-improvement",
    "self-esteem",
    "self-discipline",
    "self-respect",
    "self-confidence",
    "self-awareness",
    "self-compassion",
    "self-acceptance",
    "self-development",
    "self-motivation",
    "self-control",
    "self-growth",
    "self-belief",
    "self-worth",
    "self-help",
    "self-realization",
    "self-discovery",
    "self-reflection",
    "self-expression",
    "self-regulation",
    "self-empowerment",
    "self-actualization",
    "mindfulness",
]

## Step 4: Quote Generation and Management System

### Imports:
- **openai**: Used to interact with OpenAI's API to generate quotes using the GPT model.

- **os**: Allows accessing environment variables, particularly to retrieve the OPENAI_API_KEY which is necessary for authenticating with OpenAI’s API.

- **pandas**: Used to handle the quote database, stored as a CSV file, allowing the program to read and manage the existing quotes.

- **random**: Used to randomly select a topic for the generated quote.

### Class: QuoteGenerator
This class manages the process of generating quotes using OpenAI's GPT model, ensuring that quotes are unique by checking them against a database of existing quotes. It also stores new quotes that haven't been generated before.

### 1. `__init__(self, quotes_db_path="data/quotes.csv")`
The constructor method is responsible for:

- Initializing the API key from the environment variable `OPENAI_API_KEY`.

- Setting up the OpenAI API client (`self.openai`).

- Attempting to load the quotes database from the specified path (`quotes_db_path`).

  - If the database (`quotes.csv`) doesn't exist, it creates an empty CSV file with a column named `quote`.

### 2. `does_quote_exist(self, quote) -> bool`
This method checks if a given quote already exists in the quotes database:

- It uses pandas to check if the provided quote is contained in the `"quote"` column of the loaded database (`self.quotes_db`).

- If the quote already exists, it returns `True`; otherwise, it returns `False`.

### 3. `add_quote(self, quote)`
This method adds a quote to the database:

- It first checks if the quote already exists in the database by calling `does_quote_exist()`.

  - If the quote already exists, it prints a message and exits.

  - If the quote does not exist, it appends the quote to the dataframe and saves it back to the CSV file (`quotes.csv`).

  - A success message is printed after adding the new quote.

### 4. `get_random_topic(self)`
This method returns a random topic from a predefined list (`QUOTABLE_TOPICS`):

- The list of topics could include categories like "inspirational", "motivational", etc. It picks a random topic using `random.choice()`.

### 5. `generate_quote(self)`
This is the main method that generates a quote using OpenAI's GPT model:

- **Random Topic**: It calls `get_random_topic()` to select a random topic from the list.

- **Prompt Creation**: It creates a prompt asking the AI to generate a quote on the selected topic, while ensuring that the quote is unique and doesn't already exist in the database.

  - The prompt explicitly instructs the AI to check if the generated quote exists in the database and provides the tool (`does_quote_exist`) to validate it.

- **Tools for Checking Quote Existence**: The code specifies that the AI should use a function (`does_quote_exist`) to check if the generated quote is already in the database.

  - This is done via OpenAI's function calling mechanism, where the GPT model can call external functions as part of its response generation process.

  - The function expects to receive a parameter `quote_to_check` (the generated quote) to see if it exists in the database.

### OpenAI API Call (Function Calling)
The `openai.chat.completions.create()` method is used to make the API request to GPT-4.

- It sends:
  1. A **system message** to tell the AI that it is a quote generator.

  2. A **user message** with the selected topic.

  3. **Tools** which define a function (`does_quote_exist`) that the model can call to verify if a quote is already in the database.

- **Tool Calls**: The function calling is handled by OpenAI’s tool calling feature, where the model can make API calls to interact with external tools. The response from the model will include information on whether the generated quote already exists or not.

  - The response from the model (`completion.choices[0].message.tool_calls`) includes the result of the tool calls.

  - The tool output will inform whether the quote already exists in the database.

### 6. Tool Response Handling
The method processes the responses from the tool calls:

- It extracts the `quote_to_check` from the tool call parameters.

- It checks whether the quote already exists in the database using `self.does_quote_exist()`.

- Depending on whether the quote is found, it prints a message indicating if the quote already exists or not.

### Tool Calls:
- **Tool Definition**: The tool (`does_quote_exist`) is defined to check whether a given quote exists in the database.

  - The tool is described with:
    - `type`: "function"
    - `function`: The function definition, including its parameters (in this case, `quote_to_check`).

- **Tool Response**: Based on the output of the tool call, it prints if the quote exists or not in the database.


## Purpose:

This step defines the entire process of generating, validating, and storing unique quotes using OpenAI’s GPT model, while managing a quote database to ensure no duplicate quotes are added.

In [8]:
# Reference: https://cookbook.openai.com/examples/function_calling_with_an_openapi_spec
from openai import OpenAI
import os
import pandas as pd
import random


class QuoteGenerator:
    """A simple quote generator using LLM that demonstrates tool use."""

    def __init__(self, quotes_db_path="data/quotes.csv"):
        self.api_key = os.getenv("OPENAI_API_KEY")
        self.openai = OpenAI(self.api_key)
        try:
            self.quotes_db = pd.read_csv(quotes_db_path)
        except FileNotFoundError:
            print(f"Quotes database not found at {quotes_db_path}.")
            empty_quotes = pd.DataFrame(columns=["quote"])
            empty_quotes.to_csv(quotes_db_path, index=False)
            self.quotes_db = empty_quotes

    def does_quote_exist(self, quote) -> bool:
        """Checks if a quote exists in the database"""
        return self.quotes_db["quote"].str.contains(quote).any()

    def add_quote(self, quote):
        """Adds a quote to the database"""
        if self.does_quote_exist(quote):
            print("Quote already exists in the database.")
            return
        self.quotes_db = self.quotes_db.append({"quote": quote}, ignore_index=True)
        self.quotes_db.to_csv("data/quotes.csv", index=False)
        print("Quote added successfully.")

    def get_random_topic(self):
        """Gets a random topic from the quotable topics"""
        return random.choice(QUOTABLE_TOPICS)

    def generate_quote(self):
        """Generates a quote using the LLM model"""
        topic = self.get_random_topic()
        prompt = f"""
            You are a quote generator AI that generates quotes for the user.
            Topic: {topic}
            Generate a quote about {topic}.
            Make sure the quote is unique and doesn't exist in the database.
            You have a tool to check if the quote already exists in the database.
        """
        tools = [
            {
                "type": "function",
                "function": {
                    "name": "does_quote_exist",
                    "parameters": {
                        "type": "object",
                        "properties": {"quote_to_check": {"type": "string"}},
                        "required": ["quote_to_check"],
                    },
                },
            }
        ]
        completion = self.openai.chat.completions.create(
            messages=[
                {"role": "system", "content": "You are a quote generator AI."},
                {"role": "user", "content": topic},
            ],
            model="gpt-4o",
            tools=tools,
        )

        tool_calls = completion.choices[0].message.tool_calls

        tool_output = ""

        for tool_call in tool_calls:
            tool_output += "\n" + tool_call.name + "\n"
            if tool_call.name == "does_quote_exist":
                quote_to_check = tool_call.parameters.quote_to_check
                quote_exists = self.does_quote_exist(quote_to_check)
                if quote_exists:
                    tool_output += (
                        f"The quote '{quote_to_check}' already exists in the database."
                    )
                else:
                    tool_output += (
                        f"The quote '{quote_to_check}' does not exist in the database."
                    )
        # TODO: Finish this function

## Conclusion:

The provided code showcases a practical and interactive implementation of a quote generator using OpenAI's language model, coupled with a local database for storing and validating generated quotes. The program effectively demonstrates the following key concepts:

- `Environment Setup:` The application ensures that all necessary dependencies are installed, and environment variables, especially the OPENAI_API_KEY, are set up correctly.

- `Database Integration:` It manages a local CSV file (quotes.csv) as a database to store unique quotes, and provides functionality to check whether a newly generated quote already exists in the database.

- `Function Calling with OpenAI:` The application utilizes OpenAI’s function calling feature to interact with a tool that checks for quote uniqueness in the database, ensuring that duplicate quotes are not added.

- `Random Topic Selection:` It randomly selects from a predefined list of quotable topics, which makes the generated quotes varied and diverse, ensuring a fresh set of responses each time the system is invoked.

- `Error Handling:` The code incorporates checks to ensure smooth functioning, such as verifying the presence of environment variables and handling potential database issues like missing files.

- `User Interaction:` The quote generation and validation process are automated, with feedback provided to the user if the quote already exists, ensuring a seamless and efficient experience.

In conclusion, the application serves as a robust example of combining OpenAI's powerful GPT models with practical functionality like database management, function calling, and ensuring data uniqueness. It can easily be extended for various content generation tasks where uniqueness and accuracy are essential.


---

# Thank You for visiting The Hackers Playbook! 🌐

If you liked this research material;

- [Subscribe to our newsletter.](https://thehackersplaybook.substack.com)

- [Follow us on LinkedIn.](https://www.linkedin.com/company/the-hackers-playbook/)

- [Leave a star on our GitHub.](https://www.github.com/thehackersplaybook)

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>